# Setup

In [ ]:
# Importamos las librerias necesarias

import shutil
from PIL import Image, ImageDraw
from matplotlib.pyplot import imshow
import numpy as np
import glob
import os
import re
from datetime import datetime
from pathlib import Path

In [ ]:
!mkdir datasets

In [ ]:
def full_name_func(source_folder,dataset,file_exclusion_patterns = [],subdirectories_included = [],frames = 4):

    full_name_list = []


    for root, dirs, files in os.walk(source_folder):
        for directory in dirs:
            if directory == 'images':
                source_images_path = os.path.join(root, directory)
                for file in os.listdir(source_images_path):
                    if not file.startswith(tuple(file_exclusion_patterns)):
                        src_file = os.path.join(source_images_path, file)
                        subfolder = os.path.basename(Path(src_file).parents[1])
                        if ((subfolder in subdirectories_included) or (not subdirectories_included)):
                            full_name = dataset + '-' + subfolder +'-'+ file
                            full_name_list.append(full_name[:-4])

    full_name_list.sort()

    final_list = [file for i,file in enumerate(full_name_list) if i%frames == 0]


    return final_list

In [ ]:
def drive_to_colab(source_folder,destination_images,destination_labels,dataset,file_list):

    os.makedirs(destination_images, exist_ok=True)
    os.makedirs(destination_labels, exist_ok=True)


    for root, dirs, files in os.walk(source_folder):
        for directory in dirs:
            if directory == 'images':
                source_images_path = os.path.join(root, directory)
                for file in os.listdir(source_images_path):
                    src_file = os.path.join(source_images_path, file)
                    dst_file = os.path.join(destination_images, file)

                    subfolder = os.path.basename(Path(src_file).parents[1])
                    full_name = dataset + '-' + subfolder +'-'+ file
                    new_path = os.path.join(destination_images, full_name)

                    if full_name[:-4] in file_list:
                        shutil.copy(src_file, dst_file)
                        shutil.move(dst_file, new_path)
            elif directory == 'labels':
                source_labels_path = os.path.join(root, directory)
                for file in os.listdir(source_labels_path):
                    src_file = os.path.join(source_labels_path, file)
                    dst_file = os.path.join(destination_labels, file)

                    subfolder = os.path.basename(Path(src_file).parents[1])
                    full_name = dataset + '-' + subfolder +'-'+ file
                    new_path = os.path.join(destination_labels, full_name)

                    if full_name[:-4] in file_list:
                        shutil.copy(src_file, dst_file)
                        shutil.move(dst_file, new_path)

In [ ]:
subs = []
for i in range(102,126):
  subs.append(str(i) + 'FTASK')
# Carpetas que no existen
for item in ['115FTASK','104FTASK','107FTASK','109FTASK','112FTASK',]:
  subs.remove(item)

# Carpetas usadas en train
for item in ['106FTASK','108FTASK','113FTASK','114FTASK','119FTASK','120FTASK','122FTASK','125FTASK']:
  subs.remove(item)

In [ ]:
destination_images = '/content/datasets/DELT/images/val'
destination_labels = '/content/datasets/DELT/labels/val'

source_folder = '/content/drive/MyDrive/UOC/DELT/osaconservation/Arribada2023GT'

dataset = 'osaconservation'

n_frames = 1

s_included = subs
# s_included = []

source_folder = '/content/drive/MyDrive/UOC/DELT/' + dataset
file_list = full_name_func(source_folder,dataset,subdirectories_included = s_included,frames=n_frames)
drive_to_colab(source_folder,destination_images,destination_labels,dataset,file_list)


In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt comet_ml

import torch
import utils

display = utils.notebook_init()

YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.9/235.7 GB disk)


In [ ]:
!cp ../drive/MyDrive/UOC/DELT/DELT.yaml data
!cp ../drive/MyDrive/UOC/DELT/.comet.config .

## Osaconservation

In [ ]:
model_v = 'osaconservation-202411110929-yolov5s'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!env COMET_LOG_PER_CLASS_METRICS=true python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/osaconservation-202411110929-yolov5s/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-202411110929-yolov5s-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 2580 images, 1723 backgrounds, 0 corrupt: 100% 4303/4303 [00:04<00:00, 1058.59it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 135/135 [01:12<00:00,  1.86it/s]
        

## Osaconservation & Vivemar

In [ ]:
model_v = 'osaconservation-vivemar-202411111444-yolov5s'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!env COMET_LOG_PER_CLASS_METRICS=true python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/osaconservation-vivemar-202411111444-yolov5s/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-vivemar-202411111444-yolov5s-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val.cache... 2580 images, 1723 backgrounds, 0 corrupt: 100% 4303/4303 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 135/135 [01:17<00:00,  1.75it/s]
                   all       4303      11690      0.958     

## Osaconservation & YouTube

In [ ]:
model_v = 'osaconservation-youtubenew-202411121056-yolov5s'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!env COMET_LOG_PER_CLASS_METRICS=true python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtubenew-202411121056-yolov5s/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-youtubenew-202411121056-yolov5s-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val.cache... 2580 images, 1723 backgrounds, 0 corrupt: 100% 4303/4303 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 135/135 [01:15<00:00,  1.79it/s]
                   all       4303      11690      0.95

## Osaconservation & FloridaState

In [ ]:
model_v = 'osaconservation-floridaState-202411120851-yolov5s'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!env COMET_LOG_PER_CLASS_METRICS=true python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/osaconservation-floridaState-202411120851-yolov5s/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-floridaState-202411120851-yolov5s-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val.cache... 2580 images, 1723 backgrounds, 0 corrupt: 100% 4303/4303 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0% 0/135 [00:00<?, ?it/s]WARNING ⚠️ NMS time limit 2.100s exceeded
                 Cl

# FloridaState

In [ ]:
model_v = 'floridaState-202411151502-yolov5s-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!env COMET_LOG_PER_CLASS_METRICS=true python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/floridaState-202411151502-yolov5s-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=floridaState-202411151502-yolov5s-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:06<00:00, 651.59it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 140/14

## YouTube

In [ ]:
model_v = 'youtubenew-202411141045-yolov5s'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!env COMET_LOG_PER_CLASS_METRICS=true python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/youtubenew-202411141045-yolov5s/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=youtubenew-202411141045-yolov5s-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:02<00:00, 1951.91it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 140/140 [01:19<00:00,  1.77it/s]
                  

# Vivemar (27 epocas)

In [ ]:
model_v = 'vivemar-202411141319-yolov5s'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/vivemar-202411141319-yolov5s/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=vivemar-202411141319-yolov5s-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val.cache... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 140/140 [01:15<00:00,  1.86it/s]
                   all       4479      11959      0.446        0.4      0.293      0.136
Spe

# Youtube - 5 frames (Paciencia 5)

In [ ]:
model_v = 'youtubenew-202411150808-yolov5s-new_version'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/youtubenew-202411150808-yolov5s-new_version/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=youtubenew-202411150808-yolov5s-new_version-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 2580 images, 1723 backgrounds, 0 corrupt: 100% 4303/4303 [00:02<00:00, 1557.45it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0% 0/135 [00:00<?, ?it/s]WARNI

Creo que se tendría que haber entrenado más etapas. Es un resultado a ignorar.

# Vivemar - 5 frames (Paciencia 5)

In [ ]:
model_v = 'vivemar-202411150826-yolov5s-new_version-5_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/vivemar-202411150826-yolov5s-new_version-5_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=vivemar-202411150826-yolov5s-new_version-5_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 2580 images, 1723 backgrounds, 0 corrupt: 100% 4303/4303 [00:02<00:00, 2035.84it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 135/135 [01:12<

Creo que se tendría que haber entrenado más etapas. Es un resultado a ignorar.

# Osaconservation (con algunos subsets de Arribada2023) - 4 frames

In [ ]:
model_v = 'osaconservation-202411150910-yolov5s-new_version-4_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# Entrenado con ['PreArribada','105FTASK','111FTASK','121FTASK','124FTASK','103FTASK']

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/osaconservation-202411150910-yolov5s-new_version-4_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-202411150910-yolov5s-new_version-4_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 1980 images, 1307 backgrounds, 0 corrupt: 100% 3287/3287 [00:01<00:00, 2039.73it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0%

# Osaconservation (con algunos subsets de Arribada2023) - Todos los frames

In [ ]:
model_v = 'osaconservation-202411150936-yolov5s-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# Entrenado con ['PreArribada','105FTASK','111FTASK','121FTASK','124FTASK','103FTASK']

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/osaconservation-202411150936-yolov5s-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=True, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-202411150936-yolov5s-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 1980 images, 1307 backgrounds, 0 corrupt: 100% 3287/3287 [00:01<00:00, 2062.45it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 10

# Vivemar (4 frames - 10 paciencia)

Con 10% de val:

In [ ]:
model_v = 'vivemar-202411151054-yolov5s-new_version-4_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/vivemar-202411151054-yolov5s-new_version-4_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=vivemar-202411151054-yolov5s-new_version-4_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:02<00:00, 2005.51it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0% 0/140 [00:00<?,

Con 20% en la val:

In [ ]:
model_v = 'vivemar-202411151206-yolov5s-new_version-4_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/vivemar-202411151206-yolov5s-new_version-4_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=vivemar-202411151206-yolov5s-new_version-4_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:03<00:00, 1304.49it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 140/140 [01:14<

# Youtube (4 frames - 10 paciencia)

In [ ]:
model_v = 'youtube-202411151126-yolov5s-new_version-4_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/youtube-202411151126-yolov5s-new_version-4_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=youtube-202411151126-yolov5s-new_version-4_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:02<00:00, 1856.70it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0% 0/140 [00:00<?,

# Bahamas (full resolution)

In [ ]:
model_v = 'bahamas-202411241210-yolov5s-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/bahamas-202411241210-yolov5s-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=bahamas-202411241210-yolov5s-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-387-g4c904ac7 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:02<00:00, 2002.07it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0% 0/140 [00:00<?,

# Bahamas (talla S, 70 epocas, menor res)

In [ ]:
model_v = 'bahamas-202411250923-yolov5s-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/bahamas-202411250923-yolov5s-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=bahamas-202411250923-yolov5s-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:03<00:00, 1475.20it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 140/140 [01:19<

# Bahamas (talla S, más de 100 epocas, low res)

In [ ]:
model_v = 'bahamas-202411271246-yolov5s-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/bahamas-202411271246-yolov5s-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=bahamas-202411271246-yolov5s-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:04<00:00, 1042.54it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 140/140 [01:11<

# Bahamas (menor resolución, talla L, 150 epocas)

In [ ]:
model_v = 'bahamas-202411251318-yolov5l-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/bahamas-202411251318-yolov5l-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=bahamas-202411251318-yolov5l-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
val: Scanning /content/datasets/DELT/labels/val.cache... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 140/140 [02:19<00:00,  1.00it/s]
                   all       4479      11959    

# Bahamas (menor resolución, talla L, 138 epocas - early stopped)

In [ ]:
model_v = 'bahamas-202411252036-yolov5l-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/bahamas-202411252036-yolov5l-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=bahamas-202411252036-yolov5l-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
val: Scanning /content/datasets/DELT/labels/val.cache... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 140/140 [02:28<00:00,  1.06s/it]
                   all       4479      11959    

# Vivemar (todos los frames, más epocas)

In [ ]:
model_v = 'vivemar-202411280937-yolov5s-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/vivemar-202411280937-yolov5s-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=vivemar-202411280937-yolov5s-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:04<00:00, 957.54it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 140/140 [01:11<0

# Vivemar (frame=2)

In [ ]:
model_v = 'vivemar-202411281030-yolov5s-new_version-2_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/vivemar-202411281030-yolov5s-new_version-2_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=vivemar-202411281030-yolov5s-new_version-2_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val.cache... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 140/140 [01:13<00:00,  1.89it/s]
                   all       4479      11959      

# Florida State (más epocas)

In [ ]:
model_v = 'floridaState-202411281131-yolov5s-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/floridaState-202411281131-yolov5s-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=floridaState-202411281131-yolov5s-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val.cache... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0% 0/140 [00:00<?, ?it/s]WARNING ⚠️ NMS time limit 2.100s exceeded
         

# Florida State (frame=2)

In [ ]:
model_v = 'floridaState-202411281150-yolov5s-new_version-2_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/floridaState-202411281150-yolov5s-new_version-2_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=floridaState-202411281150-yolov5s-new_version-2_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val.cache... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 140/140 [01:13<00:00,  1.91it/s]
                   all       4479      1

# YouTube (más epocas)

In [ ]:
model_v = 'youtubenew-202411281209-yolov5s-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/youtubenew-202411281209-yolov5s-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=youtubenew-202411281209-yolov5s-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val.cache... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 140/140 [01:19<00:00,  1.75it/s]
                   all       4479      11959

# YouTube (frame=2)

In [ ]:
model_v = 'youtubenew-202411281329-yolov5s-new_version-2_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/youtubenew-202411281329-yolov5s-new_version-2_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=youtubenew-202411281329-yolov5s-new_version-2_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val.cache... 2756 images, 1723 backgrounds, 0 corrupt: 100% 4479/4479 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 140/140 [01:15<00:00,  1.86it/s]
                   all       4479      11959

# OSA Conservation (Experimento 1, todos los frames)

In [ ]:
model_v = 'osaconservation-202412020839-yolov5s-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# ['PreArribada','102FTASK','103FTASK','105FTASK','111FTASK','116FTASK','118FTASK','121FTASK','124FTASK']

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/osaconservation-202412020839-yolov5s-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-202412020839-yolov5s-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 1665 images, 1177 backgrounds, 0 corrupt: 100% 2842/2842 [00:01<00:00, 1457.73it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0%

# OSA Conservation (Experimento 2, todos los frames)

In [ ]:
model_v = 'osaconservation-202412020932-yolov5s-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# ['PreArribada','106FTASK','108FTASK','113FTASK','114FTASK','119FTASK','120FTASK','122FTASK','125FTASK']

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/osaconservation-202412020932-yolov5s-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-202412020932-yolov5s-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 1408 images, 848 backgrounds, 0 corrupt: 100% 2256/2256 [00:01<00:00, 1932.51it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 

# OSA Conservation + YouTube (Talla S)

In [ ]:
model_v = 'osaconservation-youtube-202412021450-yolov5s-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# ['PreArribada','106FTASK','108FTASK','113FTASK','114FTASK','119FTASK','120FTASK','122FTASK','125FTASK']

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412021450-yolov5s-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-youtube-202412021450-yolov5s-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 1408 images, 848 backgrounds, 0 corrupt: 100% 2256/2256 [00:01<00:00, 1997.16it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50  

# OSA Conservation + YouTube (Talla n)

In [ ]:
model_v = 'osaconservation-youtube-202412031146-yolov5n-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# ['PreArribada','106FTASK','108FTASK','113FTASK','114FTASK','119FTASK','120FTASK','122FTASK','125FTASK']

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412031146-yolov5n-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-youtube-202412031146-yolov5n-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 1408 images, 848 backgrounds, 0 corrupt: 100% 2256/2256 [00:01<00:00, 2044.18it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50   

# OSA Conservation + YouTube (Talla M)

In [ ]:
model_v = 'osaconservation-youtube-202412031757-yolov5m-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# ['PreArribada','106FTASK','108FTASK','113FTASK','114FTASK','119FTASK','120FTASK','122FTASK','125FTASK']

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412031757-yolov5m-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-youtube-202412031757-yolov5m-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
val: Scanning /content/datasets/DELT/labels/val.cache... 1408 images, 848 backgrounds, 0 corrupt: 100% 2256/2256 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 71/71 [00:39<00:00,  1.80it/s]
                   a

# OSA Conservation + YouTube (Talla L)

In [ ]:
model_v = 'osaconservation-youtube-202412032305-yolov5l-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# ['PreArribada','106FTASK','108FTASK','113FTASK','114FTASK','119FTASK','120FTASK','122FTASK','125FTASK']

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412032305-yolov5l-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-youtube-202412032305-yolov5l-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
val: Scanning /content/datasets/DELT/labels/val.cache... 1408 images, 848 backgrounds, 0 corrupt: 100% 2256/2256 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 71/71 [01:01<00:00,  1.15it/s]
                   

# OSA Conservation + YouTube (Talla X)

In [ ]:
model_v = 'osaconservation-youtube-202412040731-yolov5x-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# ['PreArribada','106FTASK','108FTASK','113FTASK','114FTASK','119FTASK','120FTASK','122FTASK','125FTASK']

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412040731-yolov5x-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-youtube-202412040731-yolov5x-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val.cache... 1408 images, 848 backgrounds, 0 corrupt: 100% 2256/2256 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 71/71 [02:00<00:00,  1.69s/it]
                   

# OSA + Vive Mar (Mejorado)

In [ ]:
model_v = 'osaconservation-vivemar-202412091636-yolov5s-new_version-1_frames'
weigths_path = '/content/drive/MyDrive/UOC/DELT/weights/'+model_v+'/weights/best.pt'

project_name = model_v + '-val'
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# ['PreArribada','106FTASK','108FTASK','113FTASK','114FTASK','119FTASK','120FTASK','122FTASK','125FTASK']

!python val.py --data data/DELT.yaml --weights {weigths_path} --project {project_folder} --name {project_name}

val: data=data/DELT.yaml, weights=['/content/drive/MyDrive/UOC/DELT/weights/osaconservation-vivemar-202412091636-yolov5s-new_version-1_frames/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-vivemar-202412091636-yolov5s-new_version-1_frames-val, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/datasets/DELT/labels/val... 1408 images, 848 backgrounds, 0 corrupt: 100% 2256/2256 [00:01<00:00, 1900.06it/s]
val: New cache created: /content/datasets/DELT/labels/val.cache
                 Class     Images  Instances          P          R      mAP50  

In [ ]:
from google.colab import runtime
runtime.unassign()